# Python code of Gibbs sampling model for family test result Covid problem

In [157]:
# import neccessary libraries
import numpy as np
import random
import sympy as sym
from joblib import Parallel, delayed
from sympy import *
from sympy import symbols
from sympy.plotting import plot, plot3d
from datetime import datetime
from scipy.stats import beta
import winsound
duration = 1000  # milliseconds
freq = 440  # Hz

In [158]:
# function to count the number of the infected neighbores of i at t:
def CNbr(G,X,n,T):
    C=np.zeros((T,n))
    for t in range(T):
        C[t]=G[t].dot(X.T[t])
    return C.T

In [159]:
# Function to obtain the very initial sample of X, using forwad sampling:
def Forward_Sampling(T,n,G,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4] 
    x=int((1-P)*n)
    X=np.zeros((n,T))  
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    infR=CNbr(G,X,n,T)
    for t in range(T-1):
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c
                p1=alpha_+beta_*c
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 
    return X

In [160]:
def Sample_hidden_state(T,n,X,G,Y,param,t):
    
    alpha_=param[0]
    beta_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]
    
    pow_stay_healthy,pow_become_infected,pow_recovery,pow_stay_infected=0,0,0,0
    
    if (t==0):
        c=G[t].dot(X.T[t])
    else:    
        c=G[t-1].dot(X.T[t-1])
        
    for i in range(n):
    
        pow0=np.count_nonzero(Y[i,t] == 0)  
        pow1=np.count_nonzero(Y[i,t] == 1)
    
        p0=(1-theta_0_)**pow0
        p1=(1-theta_1_)**pow0
    
        p0=(theta_0_)**pow1
        p1=theta_1_**pow1
        
        if (X[i,t]==1)&(t==0):
            c0=c[i]-1
            c1=c[i]
        else:
            c0=c[i]
            c1=c[i]+1
        
        stay_healthy0= 1-alpha_-beta_*c0
        become_infected0=alpha_+beta_*c0
    
        stay_healthy1=1-alpha_-beta_*c1
        become_infected1=alpha_+beta_*c1
        
        if t==T-1:
            if X[i,t-1]==0:
                p0=p0*stay_healthy0
                p1=p1*become_infected0
            else:
                p0=p0*gama_
                p1=p1*(1-gama_) 
        else:        
            state_transition=X.T[t]+2*X.T[t+1]+1
            key=np.multiply(G[0][i],state_transition)
    
            pow_stay_healthy=pow_stay_healthy+np.count_nonzero(key == 4)
            pow_become_infected=pow_become_infected+np.count_nonzero(key == 3)
            pow_recovery=pow_recovery+np.count_nonzero(key == 2)
            pow_stay_infected=pow_stay_infected+np.count_nonzero(key ==1)
    
            p0=P*p0*(stay_healthy0)**pow_stay_healthy*(become_infected0)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            p1=(1-P)*p1*(stay_healthy1)**pow_stay_healthy*(become_infected1)**pow_become_infected*(gama_)**pow_recovery*(1-gama_)**pow_stay_infected
            
        if p0+p1==0:
            l=0.5
        else:
            l=p1/(p0+p1)
        X[i,0]=np.random.binomial( 1,  l,size=None)    
        
    return X    

In [161]:
# Gibbs sampling to obtain X, as new sample of posterior distribution:
def Calculate_X(K,T,n,X,G,Y,param):
    for k in range(K):
        for t in range(T):
            X=Sample_hidden_state(T,n,X,G,Y,param,t)
    return X                

In [162]:
# function to sample new parameters and update parameters:
def Params(R,G,X,n,T,Y,param):
    
    alpha_=param[0]
    bata_=param[1]
    gama_=param[2]
    theta_0_=param[3]
    thata_1_=param[4]    
   
    TP=np.sum(np.multiply(X,Y))
    FP=n*T-np.count_nonzero(X-Y+1)
    
    infR=np.array(CNbr(G,X,n,T))
    
    alpha_=Sample_alpha(a_alpha + n*T- np.count_nonzero(R) , b_alpha - np.count_nonzero(X)+ np.count_nonzero(R))
    beta_=Sample_beta(a_beta + n*T-np.count_nonzero(R-2) , b_beta +np.sum(np.multiply((1-X),infR))-n*T+np.count_nonzero(R-2))
    gama_=Sample_gama(a_gama +(T-1)*n-np.count_nonzero(X[:,:-1]-X[:,1:]-1), b_gama+np.sum(X)-(T-1)*n+np.count_nonzero(X[:,:-1]-X[:,1:]-1))
    theta_0_=Sample_theta0( a_teta0+FP,b_teta0+n*T-np.count_nonzero(X)-FP)
    print(a_teta1+TP,b_teta1+np.count_nonzero(X)-TP)
    theta_1_=Sample_theta1( a_teta1+TP,b_teta1+np.count_nonzero(X)-TP)
    
    R=np.zeros((n,T))+1
    for i in range(n):
        for t in range(T-1):
            infr=int(infR[i,t])
            pr_a=alpha_/(alpha_+beta_*infr)
            pr_b=beta_/(alpha_+beta_*infr)
            v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
            if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2
    param.append([alpha_,beta_,gama_,theta_0_,theta_1_])
    return param,R

# Functions to sample from beta distribution


In [163]:
def Sample_alpha(a_alpha, b_alpha):
    for i in beta.rvs(a_alpha, b_alpha, size=1000):
        if (i>0.001)&(i<0.051):
            alpha_=round(i,3)
            break
    return alpha_        


In [164]:
def Sample_beta(a_beta, b_beta):
    for i in beta.rvs(a_beta, b_beta, size=1000):
        if (i>0.0001)&(i<0.051):
            beta_=round(i,4)
            break
    return beta_        


In [165]:
def Sample_gama(a_gama,b_gama):
    for i in beta.rvs(a_gama, b_gama, size=1000):
        if (i>0.1)&(i<0.7):
            gama_=round(i,3)
            break
    return gama_  


In [166]:
def Sample_theta0(a_teta0, b_teta0):
    for i in beta.rvs(a_teta0, b_teta0, size=10000):
        if (i>0.001)&(i<0.3):
            theta_0_=round(i,3)
            break
    return theta_0_  


In [190]:
def Sample_theta1(a_teta1, b_teta1):
    for i in beta.rvs(a_teta1, b_teta1, size=10000):
        print(i)
        if .9901>i>0.8:
            theta_1_=round(i,3)
            break
    return theta_1_  


In [171]:
# initialize parameters for beta distributions:
a_alpha=.1
b_alpha=7
a_beta=2
b_beta=1
a_gama=1
b_gama=3
a_teta0=1
b_teta0=5
a_teta1=800
b_teta1=1
p_=.8

In [172]:
#Sample infection and emision parameters(alpha,beta,gama,teta0,teta1)
alpha_=Sample_alpha(a_alpha, b_alpha)
beta_=Sample_beta(a_beta, b_beta)
gama_=Sample_gama(a_gama,b_gama)
theta_0_=Sample_theta0(a_teta0, b_teta0)
theta_1_=Sample_theta1(a_teta1, b_teta1)

In [183]:
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_])
params

[[0.023, 0.0199, 0.229, 0.022, 0.99]]

In [184]:
# Function to generates Synthetic dataset
def Synthetic_Data(n,T,y,params):
    alpha_,beta_,gama_,theta_0_,theta_1_=params[0],params[1],params[2],params[3],params[4]
    x=int((1-p_)*n)

    X=np.zeros((n,T))
    idx=np.random.choice(range(n), x)
    X[idx,0]=1
    # Random social network
    G=[]
    for j in range(T):
        g=np.identity(n)
        for i in range(n):
            inx=np.random.choice(range(i,n), y)
            g[i,inx]=1  
            g[inx,i]=1
        G.append(g)
    G=np.array(G)
    infR=CNbr(G,X,n,T)
    # Synthetize X, using params,G and transition probability:
    for t in range(T-1):
        for i in range(n):
            c=infR[i,t]
            if X[i,t]==0:
                p0=1-alpha_-beta_*c
                p1=alpha_+beta_*c
            else:
                p0=gama_
                p1=1-gama_
            if p0+p1==0:
                l=0.5
            else:
                l=p1/(p0+p1)
            X[i,t+1]=np.random.binomial( 1, l,size=None) 

    # Synthetize Y, using params,G, X, emission probability:
    Y=np.zeros((n,T))
    for t in range(T):
        for i in range(n):
            if X[i,t]==0:
                Y[i,t]=np.random.binomial( 1, theta_0_,size=None) 
            else:
                Y[i,t]=np.random.binomial( 1, theta_1_,size=None) 
    return G,Y,X            

In [185]:
# Generate synthetic data,G ,Y:
n,T,y=100,100,10
synthetic_data=Synthetic_Data(n,T,y,params[-1])
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

In [186]:
# Save true value of X in Z, as the correct label of data:
Z=X
np.sum(Z)

915.0

In [187]:
np.array(params)

array([[0.023 , 0.0199, 0.229 , 0.022 , 0.99  ]])

In [188]:
# Run Gibbs sampling algorithm to estimate X:
U=10
J=1000
P=p_
X=Forward_Sampling(T,n,G,params[-1])
X=Calculate_X(J,T,n,X,G,Y,params[-1])
winsound.Beep(freq, duration)

In [189]:
# define auxiliary variable R(n,t):
R=np.zeros((n,T))+1
infR=np.array(CNbr(G,X,n,T))
for i in range(n):
    for t in range(T-1):
        infr=int(infR[i,t])
        pr_a=alpha_/(alpha_+beta_*infr)
        pr_b=beta_/(alpha_+beta_*infr)
        v=np.random.multinomial(1, [pr_a]+[pr_b]*infr)
        if (X[i][t]==0)&(X[i][t+1]==1):
                if v[0]==1:
                    R[i,t]=0
                else: 
                    R[i,t]=2

In [191]:
# Main code to run entire Gibbs algorithm U times:
for i in range(U):
    print("******************* Iteration:",i,"*****************************************************************************")
    now = datetime.now()
    current_time = now.strftime("%H:%M:%S")
    print("Current Time is :", current_time)
    prm=Params(R,G,X,n,T,Y,params[-1])
    params.append(prm[0])
    X=Calculate_X(J,T,n,X,G,Y,params[-1])
    print("Verctor of Health States:","\n",X)
    R=prm[1]
winsound.Beep(freq, duration)            

******************* Iteration: 0 *****************************************************************************
Current Time is : 16:34:46
918.0 818.0
0.5312989106200171
0.5206284599255382
0.5388811690592379
0.5085009573824216
0.5357643324394802
0.5198925343511369
0.5277834210223382
0.5166715158434126
0.5280145509894238
0.5288416923244043
0.5211877519662599
0.5128366632570008
0.5243372157594823
0.5285170457827367
0.5302415318686143
0.5270016118887711
0.5130181047181416
0.5415294360986311
0.5436033712939015
0.5267560712053071
0.5186929850150774
0.5273798251231532
0.5171532685868659
0.5285690575729919
0.5188991845139224
0.5331019044426323
0.5306983361172333
0.5330599050028573
0.5240296307733824
0.5229722379603119
0.5368026390515758
0.5241941910011341
0.5258038916806619
0.5136138611661055
0.5453073136441071
0.5396090357333289
0.5505621822578902
0.5117158080286022
0.5502401045652617
0.5258559459072863
0.5446359089866439
0.547181282139596
0.5467512023830924
0.525382033185199
0.52557556634911

0.5246824905699935
0.5098327856924888
0.546672349728405
0.547353090571076
0.5361089255209538
0.532084287661477
0.5471101500730601
0.5113936714992178
0.5193016073895992
0.5193933327347507
0.5013845453210883
0.530194335545979
0.5310234691535688
0.5032888755446515
0.5304692071342032
0.5074265462828472
0.5388709678574733
0.5210046824445344
0.5140954309733553
0.5323223422812624
0.5113909825295629
0.5181439843702067
0.5245873619972398
0.5345757323449261
0.5094937485940604
0.5366939340042143
0.5511021968450274
0.528149604071049
0.5443456787364815
0.531661678387725
0.5412820756121542
0.536959164317784
0.5489374471062995
0.5082147451878916
0.5325831688205097
0.5410676403477727
0.5459168739644484
0.5295426809753261
0.5463191340899529
0.5151679637947035
0.5169154415678183
0.533044872388577
0.5116819986001071
0.5310513619491686
0.5339113701899959
0.5210603919374172
0.5297161399917617
0.5376813635019604
0.5185556156357823
0.5357245579077871
0.5184675128048375
0.5395084697773832
0.5258030987162235
0

0.5162436639150297
0.5327434265799753
0.5252369241586675
0.5323388263530209
0.5378280164092363
0.5228841376433367
0.5234245524302333
0.5482416127019853
0.5274287671706303
0.5386552417378874
0.5389938313186197
0.5258802375198087
0.5268081992895305
0.49815695548847816
0.5372076347271553
0.5469689637533903
0.5493142427657435
0.5395918609933521
0.5573341190498919
0.5318337592144334
0.5353854759824824
0.5285994229207017
0.5449213228018286
0.5203105513228286
0.553311261763863
0.5256837693188193
0.522509783050637
0.5438418931041598
0.5245828954946925
0.5272195054402075
0.5336770628996563
0.5510903514309784
0.5413496167075713
0.5483466899548566
0.5296918806907512
0.5308313924041516
0.5343987080903734
0.5239945600942066
0.5136138465478972
0.5350133277730631
0.5461317657592984
0.5275500963200208
0.5375285851969162
0.5481442665563985
0.5150122202417371
0.5340957519949583
0.5300787658184825
0.5405555004255868
0.5304755160188901
0.5350537335374647
0.5300402147645781
0.5462914571898833
0.52523333237

0.5481208638027969
0.5368953833311295
0.5132055039254619
0.5257853162276271
0.523391355454774
0.5317489263746501
0.5356773559337018
0.5295083852037507
0.5371180400248986
0.5198971097905329
0.5275796479730974
0.532017409330758
0.5268400224640063
0.5384927228682772
0.5023601763402232
0.5244808202613955
0.5337862277517404
0.487986019315898
0.5044286847205982
0.5352275144245218
0.5137429216802939
0.5115464400016643
0.5527103703697273
0.5421325350319324
0.5247409152871502
0.524494109723823
0.53396919931674
0.5157880417870211
0.5336149292413538
0.5200622985539647
0.5168344472467067
0.5402774106332952
0.5303039368463294
0.5298033307843116
0.521742717760617
0.5386858730858691
0.5261741272491812
0.5217266804232793
0.5410368539862874
0.535829233246562
0.535549988681438
0.5485486388875114
0.5130630448556417
0.5515965454306865
0.5155363347977135
0.5114227632189174
0.5241304687245901
0.5418598550436629
0.5375714125055779
0.5413050932501421
0.526366090629416
0.5273071148794338
0.5426100064585714
0.5

0.5343248200818713
0.5431060677926729
0.5292018886301232
0.5314500349304594
0.5145378637191312
0.5516483519075754
0.52446501537198
0.5159270786255641
0.5389388464805124
0.5456933769262119
0.5171600999143494
0.5311998209965756
0.5409905645079063
0.5374802086166821
0.5401505106040044
0.5254162067898835
0.5209639779024955
0.5074715277867506
0.5311024896750547
0.5147949464538235
0.5421667345470451
0.5089819749633716
0.5180583073505085
0.525644110151398
0.5315881119341218
0.5123091820962126
0.5319524700988941
0.5189383434454727
0.5378337834480483
0.5258545998283269
0.5299486809002325
0.5229493892162982
0.5402593930361411
0.5128926214653126
0.5213730742306486
0.5355069964888075
0.5489717405074489
0.5412090959275678
0.516812712542534
0.541683244481789
0.5283173890450233
0.51856091702825
0.5300258754016586
0.5135013494720043
0.51087429300475
0.5378285428024148
0.5188406987870807
0.538940701132599
0.546145485693626
0.5100173780057631
0.5343872600260113
0.5054892587211245
0.5410941883481348
0.54

0.5373116479527534
0.5295839264848554
0.5288170499740232
0.524784975789177
0.5513618598375519
0.5234975835333581
0.5256833285221247
0.5285414720461905
0.5431254942464666
0.5267714353785276
0.5509933562688492
0.5554395196898179
0.5182230851344846
0.5302581086548619
0.5273209217672885
0.5443357820830828
0.5485972612483803
0.5266095208734682
0.5375911833634743
0.5028771717804771
0.5507947994028856
0.5275001227775511
0.5244090926456167
0.5194350637893645
0.5230574357267034
0.5194571263945575
0.542012780042863
0.5456540940157986
0.5516420210380474
0.5311895986848583
0.5061612407399885
0.5416707279438663
0.519698623134702
0.5518827829329884
0.5210669106487451
0.5257399272033073
0.5348019665879834
0.5101939373038754
0.5309892474727014
0.523087515087841
0.5340763137984451
0.5326513454910228
0.5248533283922404
0.5363287382527543
0.5271303421479607
0.5387007392656665
0.533234422638935
0.5227360289721937
0.5274438519159688
0.5298025392318468
0.5185045017245565
0.5498426073893772
0.510755782130049

0.5192524372188377
0.5266164383973391
0.5273386416181777
0.5298005746970418
0.5406649394315359
0.5266089612425562
0.5166991579386225
0.5115897714962229
0.5268806731282135
0.543840262360755
0.5323701646227802
0.5253362424956929
0.500443812896166
0.526604303587148
0.528817309673926
0.5175818755549981
0.5227729093903597
0.5317301302418963
0.543712501896253
0.5125605822251605
0.5223907087003966
0.5352186530925029
0.516326569924388
0.5219310321767088
0.5394835456849363
0.5444570044171324
0.5296788003155011
0.5417412386338202
0.5109601340008324
0.5280816052528375
0.5280901784444179
0.5295402570881095
0.5225351565719081
0.5259807459168189
0.506692233940606
0.5048988578721031
0.5335808267334663
0.5212857549207133
0.5333179580793158
0.5200112103723467
0.5213546215373447
0.5306607047190585
0.5213400501429735
0.5135012808061399
0.5220242880357239
0.5388431468258534
0.5130256767527782
0.5439795297689891
0.542781760023391
0.5352148261288379
0.5446303892706081
0.5357979634361393
0.5295343112473792
0

0.5188449696778538
0.5532248759114148
0.5435357749653774
0.517707511432583
0.5187144544989594
0.5196092531752233
0.5491799747307613
0.5548676542314799
0.5209983737549583
0.5287868126749663
0.5241753770573979
0.5168045258161413
0.5565209118610691
0.5410323967091931
0.5259769348899469
0.5348921129461598
0.5357243692616248
0.5241116718828408
0.5348284696300175
0.519100715553997
0.5293660297576008
0.49687127599369674
0.5163286816293269
0.5191919219880278
0.5397131313625665
0.509785730068139
0.5382198399732785
0.5329919831935418
0.5582994671835335
0.5157919964569583
0.5305875346655348
0.5162145863757996
0.5319545106642766
0.5242531957100091
0.512275930144526
0.5251353586969884
0.5317720816754719
0.531676595239706
0.5414568183518194
0.5276444429709863
0.5354153497235904
0.5482297857258104
0.5131786458627356
0.5245784332027007
0.5448069384857083
0.5166010430793004
0.5496221224603681
0.5392257778853073
0.5270555618047601
0.5388658274099478
0.5334561207362155
0.5315774534193001
0.53873574439241

UnboundLocalError: local variable 'theta_1_' referenced before assignment

In [77]:
Sample_theta1(1028 ,316)

0.7624422120173191
0.7783112008210622
0.7563015049573965
0.7778811052560178
0.7815266339447177
0.7745504185347082
0.7725317310193937
0.7602496238811157
0.771732830930457
0.7658943951319142
0.7761891041785648
0.770779189142049
0.7627149282440279
0.7697765870377024
0.7522559662982958
0.7601289226942612
0.7824509955798324
0.7603410092121046
0.7547778348411608
0.7706800707187175
0.7853300636495828
0.7771217349452416
0.733994778631511
0.7533305152955033
0.7596094720741149
0.7690784737794445
0.7550018412668565
0.7699803216699791
0.7756163456858333
0.764478650662779
0.7613153758116493
0.76235047771541
0.7691508198607929
0.7800655165275877
0.7684795239992677
0.7743064263735142
0.7657380531184442
0.7769240122785451
0.7675306277873643
0.7485257668925959
0.7679979039376075
0.7610404419185888
0.7646309646549613
0.7571435667768172
0.7649911581699025
0.7507802668875125
0.7741510082812901
0.7668799966767141
0.7578067939779994
0.7671093155291042
0.7900972585346394
0.7792249748463735
0.759407720492849


0.804

In [ ]:
# Compare X, Z and estimate accuracy of the model:

In [ ]:
# synthetized observation vectore:
Y_=np.zeros((n,T))
for t in range(T):
    for i in range(n):
        if X[i,t]==0:
            Y_[i,t]=np.random.binomial( 1, theta_0_,size=None) 
        else:
            Y_[i,t]=np.random.binomial( 1, theta_1_,size=None) 

# ##############################################################################################################################

In [ ]:
# The following is for family test result problem:

In [148]:
# Randomly initialize params:
number_families=30
alpha_=0.005
beta_=0.045 
gama_=0.3
theta_0_=0.01
theta_1_=0.9
p_=0.9
n,T=100,100
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

# Synthetize data for famity test result problem

In [ ]:
# Choose some params close to obtimized params:
alpha_= 0.05
beta_= 0.01
gama_= 0.3
theta_0_= 0.07
theta_1_= 0.8
p_= 0.8
params=[]
params.append([alpha_,beta_,gama_,theta_0_,theta_1_,p_])

In [149]:
# Synthetize the "Family" matrix, which denotes that each two individuals are in the same family or not:
a=list(range(n))
Family=np.identity(n)
for i in range(1,number_families):
    number_family_members=np.random.randint(2, 5)
    inx=random.sample(list(a), number_family_members)
    a=set(a).difference(inx)
    for j in inx:
        for k in inx:
            Family[j,k]=1
Family

array([[1., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 0., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 1., 0., 0.],
       [0., 0., 0., ..., 0., 1., 0.],
       [0., 0., 0., ..., 0., 0., 1.]])

In [150]:
# Synthetize G,Y,X:
n,T,y=100,100,10
synthetic_data=Synthetic_Data(n,T,y,params[-1])
G,Y,X =synthetic_data[0],synthetic_data[1],synthetic_data[2]

In [151]:
# synthetize family test result matrix,YF:
unique_rows = np.unique(Family, axis=0)
unique_rows.shape
YF=np.zeros((n,T))
YF=np.dot(unique_rows,Y)
YF.shape

(50, 100)

# Simulation data is ready now, YF,G,Family:

#  Now we have sudo observation matrix Y_ and social network G, so  we do estimate X and parameters for these synthetic dataset:


In [124]:
# input: Family,YF,G

In [156]:
Y=np.zeros((n,T))
for i in range(n):
    for j in range(unique_rows.shape[0]):
        if unique_rows[j,i]==1:
            Y[i,:]=YF[j,:]

(46, 100)